In [ ]:
import numpy as np
from pso_algorithm import PSO
import matplotlib.pyplot as plt
import qutip
from qutip import expect, fidelity, qutrit_basis, qutrit_ops

import sys
sys.path.append('../')
from common.common import tsoa_coupled_spins_quantum_simulation
import threading

* Define the control process as an objective function which should be minimized
* Input params will be the params of the trigonometric series
* we will use 3 harmonics, so we need 2*3 + 2 = 8 params

In [3]:
nb_harmonics = 5
nb_params = 2 * (2 * nb_harmonics + 1)
T = 2.2
max_steps = 200
ξ = 1

In [4]:
def get_concurrence(state):
    c1 = state[0][0]
    c2 = state[1][0]
    c3 = state[2][0]

    return abs(2* c1 * c3 - c2**2)

* Parameters of the system

In [5]:
upper_bound = 1 * np.ones(nb_params)
lower_bound = -1 * np.ones(nb_params)

* PSO algorithm parameters

In [6]:
nb_particles = 300
max_iter = 1000
w_max = 0.9 # inertia weight max
w_min = 0.2 # inertia weight min
c1 = 1
c2 = 3
max_velocity = (upper_bound - lower_bound) * 0.2
min_velocity = -max_velocity

In [7]:
def training(T, max_control):
    def tsoa_objective_function(input_params):
      penalty = 2000

      basis3lvl = qutrit_basis()
      initial_quantum_state = basis3lvl[0]

      states, omegas, detunings = tsoa_coupled_spins_quantum_simulation(input_params=input_params, 
                                    initial_quantum_state=initial_quantum_state,
                                    final_time=T, nb_harmonics=nb_harmonics, max_steps=max_steps, ξ=ξ)

      created_quantum_state = states[-1]

      # add constraints
      # bound omega and detuning
      max_omega = max(omegas)
      min_omega = min(omegas)
      max_detuning = max(detunings)
      min_detuning = min(detunings)

      # define constraints for the values of the controls
      constr1 = max_omega <= max_control and min_omega >= -max_control
      constr2 = max_detuning <= max_control and min_detuning >= -max_control
      
      inconcurrence = 1 - get_concurrence(created_quantum_state)

      if constr1 and constr2 and inconcurrence > 0:
            return inconcurrence
      else: 
            return inconcurrence + penalty

    pso = PSO(nb_particles=nb_particles, nb_params=nb_params, 
          lower_bound=lower_bound, upper_bound=upper_bound,
          objective_fn=tsoa_objective_function, w_max=w_max, w_min=w_min, 
          c1=c1, c2=c2, max_velocity=max_velocity, min_velocity=min_velocity)

    pso.optimize(max_iter=max_iter)

    return pso

* PSO algorithm execution

In [8]:
concurrences_array = []
states_arrray = []
controls_array = []
global_objectives_array = []
optimal_params_array = []

In [ ]:
timings = [(2.35, 1.), (1.3, 2.), (1.2, 3.), (1.0, 4.)]
threads = []

for (T, max_control) in timings:
    pso = training(T, max_control)

    optimal_params = pso.global_best.position
    optimal_params_array.append(optimal_params)
    omega_params = optimal_params[0:2 * nb_harmonics + 1]
    detuning_params = optimal_params[2 * nb_harmonics + 1: len(optimal_params)]

    initial_quantum_state = qutrit_basis()[0]
    target_quantum_state = qutrit_basis()[1]

    states, omegas, detunings = tsoa_coupled_spins_quantum_simulation(input_params=optimal_params, initial_quantum_state=initial_quantum_state,
                                                final_time=T, nb_harmonics=nb_harmonics,max_steps=max_steps, ξ=ξ)

    concurrences = [get_concurrence(state) for state in states]
    concurrences_array.append(concurrences)
    
    states_arrray.append(states)
    controls_array.append((omegas, detunings))
    global_objectives_array.append(pso.get_global_objectives())

* Plot global best solution evolution

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
fig.set_figheight(8)
fig.set_figwidth(12)

axes = [ax1, ax2, ax3, ax4]
subplot_params = [(.35, .56, .1, .1), (.60, .65, .1, .1), (.20, .25, .1, .1), (.60, .15, .1, .1)]
titles = ['a', 'b', 'c', 'd']

for i, ax in enumerate(axes):
    omegas, detunings = controls_array[i]
    T, max_control = timings[i]
    time = np.linspace(start = 0, stop = T, num = max_steps)
    
    ax.plot(time, omegas, label = 'Ω')
    ax.plot(time, detunings, label = 'Δ')
    ax.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
    ax.set_xlabel(r"t ($\xi^{-1}$)")
    ax.set_ylim((-(max_control + 0.1), (max_control + 0.1)))
    ax.set_title('(' + titles[i] + ')', loc = "right", fontsize = 10)
    ax.legend()

    l, b, h, w = subplot_params[i]
    concurrences = concurrences_array[i]
    ax5 = fig.add_axes([l, b, w, h])
    ax5.plot(time, concurrences)
    ax5.set_ylabel("Concurrence", rotation = 90)

In [ ]:
concurrences = concurrences_array[0]
states = states_arrray[0]
omegas, detunings = controls_array[0]
global_objectives = global_objectives_array[0]

[proj1, proj2, proj3, trans12, trans23, _] = qutrit_ops()
T, max_control = timings[0]
time_span = np.linspace(start = 0, stop = T, num = max_steps)

population1 = expect(proj1.dag() * proj1, states)
population2 = expect(proj2.dag() * proj2, states)
population3 = expect(proj3.dag() * proj3, states)

max_concurrence = round(np.max(concurrences), 4)

fig, ((ax1, ax2), (ax4, ax3)) = plt.subplots(2, 2)

fig.set_figheight(8)
fig.set_figwidth(12)
# fig.suptitle(f"Trigonometric Series - {nb_harmonics} harmonics - Concurrence {max_concurrence}")

ax1.plot(time_span, omegas, label = 'Ω')
ax1.plot(time_span, detunings, label = 'Δ')
ax1.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
ax1.set_xlabel(r"t ($\xi^{-1}$)")
ax1.set_ylim((-1.1, 1.1))
ax1.legend()

ax2.plot(time_span, concurrences)
ax2.axhline(y = 0.9999, color = 'r', linestyle = '--', label = '0.9999')
ax2.set_ylabel("Concurrence", rotation = 90, fontsize = 12)
ax2.set_xlabel(r"t ($\xi^{-1}$)")
ax2.legend(loc = 'lower right')

ax4.plot(time_span, population1, label = r"$P_1$")
ax4.plot(time_span, population2, label = r"$P_2$")
ax4.plot(time_span, population3, label = r"$P_3$")
ax4.set_ylabel("Populations", rotation = 90, fontsize = 12)
ax4.set_xlabel(r"t ($\xi^{-1}$)")
ax4.legend()

ax3.plot(range(len(global_objectives)), global_objectives, label = "Objective")
ax3.set_ylabel("Objective")
ax3.set_xlabel("iteration")
ax3.set_yscale("log")
ax3.legend()

In [ ]:
np.round([(states_arrray[i][-1].full()[0], states_arrray[i][-1].full()[1] / np.sqrt(2), states_arrray[i][-1].full()[1] / np.sqrt(2), states_arrray[i][-1].full()[2]) for i in range(4)], 4)

In [ ]:
np.round(np.abs([(states_arrray[i][-1].full()[0], states_arrray[i][-1].full()[1] / np.sqrt(2), states_arrray[i][-1].full()[1] / np.sqrt(2), states_arrray[i][-1].full()[2]) for i in range(4)])**2, 4)

In [ ]:
optimal_params_array[0]

In [ ]:
concurrences